# Bringing up the Korean language ability of sLLM

AI hub에 한국어-영어 번역(병렬) 말뭉치 데이터 중 일부분을 사용하여 전처리 후 unsloth을 이용해서 fine tunning해서 만든 모델 코드 입니다.

In [24]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",    
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None,
)

In [27]:
from datasets import load_dataset
def load_parquet_data(file_path):
    dataset = load_dataset("parquet", data_files=file_path, split="train")
    dataset = dataset.train_test_split(train_size=0.001)['train']
    # return dataset['train'], dataset['test']
    return dataset

def preprocess_function(examples):
    prompt = "다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: {source}\n\n영어:"
    eos_token = tokenizer.eos_token
    return {
        "text": [prompt.format(source=src) + tgt + eos_token
                 for src, tgt in zip(examples["source_text"], examples["target_text"])],
        "target": examples["target_text"]
    }



In [28]:
parquet_file_path = "/kaggle/input/ai-hub-dataset/combined_data.parquet"

dataset= load_parquet_data(parquet_file_path)

In [29]:
dataset[0]

{'SID': 124201.0,
 'source_text': '서류를 검토한 결과 문제가 없는 것 같아요.',
 'target_text': 'We have examined the documents and everything seems to be fine.',
 'ID': None,
 '날짜': None,
 '자동분류1': None,
 '자동분류2': None,
 '자동분류3': None,
 'URL': None,
 '언론사': None,
 '키워드': None,
 '지자체': None}

In [30]:
dataset = dataset.map(preprocess_function, batched = True,)

Map:   0%|          | 0/929 [00:00<?, ? examples/s]

In [31]:
dataset[0]

{'SID': 124201.0,
 'source_text': '서류를 검토한 결과 문제가 없는 것 같아요.',
 'target_text': 'We have examined the documents and everything seems to be fine.',
 'ID': None,
 '날짜': None,
 '자동분류1': None,
 '자동분류2': None,
 '자동분류3': None,
 'URL': None,
 '언론사': None,
 '키워드': None,
 '지자체': None,
 'text': '다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: 서류를 검토한 결과 문제가 없는 것 같아요.\n\n영어:We have examined the documents and everything seems to be fine.<eos>',
 'target': 'We have examined the documents and everything seems to be fine.'}

In [32]:
FastLanguageModel.for_inference(model) 
inputs = tokenizer(
[
    "다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: "+dataset[0]['source_text']
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: 서류를 검토한 결과 문제가 없는 것 같아요.\n\n영어: The documents have been reviewed and it seems there are no problems.\n\n다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: 학교에서 학교를 졸업한 후에 학교를 졸업한 후에 학교를 졸업']

## fine-tune the Gemma model

In [33]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os

os.environ["WANDB_DISABLED"] = "true"


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20, # Set this for 1 full training run. 30 num
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/929 [00:00<?, ? examples/s]

In [34]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 929 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,320
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,2.546400
2,2.663800
3,2.550700
4,2.432100
5,2.153400
6,2.090900
7,2.058200
8,1.989300
9,1.661100
10,1.733800


In [35]:

FastLanguageModel.for_inference(model) 
inputs = tokenizer(
[
    "다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: "+dataset[0]['source_text']
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>다음 문장을 한국어에서 영어로 번역하세요:\n\n한국어: 서류를 검토한 결과 문제가 없는 것 같아요.\n\n영어: After reviewing the documents, I think there is no problem.<eos>']

### Saving models



In [36]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")


('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')